# 출력 파서(`OutputParser`)

LLM의 출력을 더 유용, 구조화된 형태로 변환함

1. 구조화: LLM의 자유분방한 형식의 출력을 JSON, dict등의 형태로 **변환**할 수 있다. -> 쓰기 편함
2. 일관성: 항상 일관된 출력형식을 주기 때문에, 후속 처리에 용이함
3. 유연성: 다양한 출력 형식이 존재. JSON, list, dict 등등 다 가능함

In [12]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()
llm = ChatOpenAI(model='gpt-4.1-nano')

## PydanticOutputParser

In [8]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [11]:
# 일반적인 이메일 형식, input
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [20]:
prompt = PromptTemplate.from_template("""
아래 이메일에서 핵심 정보를 추출해 줘
---
{email_conversation}
""")

print(prompt.format(email_conversation=email_conversation))


아래 이메일에서 핵심 정보를 추출해 줘
---
From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션




In [21]:
chain = prompt | llm
res = chain.invoke({'email_conversation': email_conversation}).content
print(res)  # 사람용 요약으로는 훌륭하지만, 구조화된 답변은 아님. 모든 이메일을 정해진 포맷에 맞게 정리하고 싶다면?

아래는 이메일의 핵심 정보입니다:

1. 발신인: 김철수 상무(바이크코퍼레이션)
2. 수신인: 이은채 대리(티디인터내셔널)
3. 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. 요청 내용:
   - "ZENESIS" 모델의 상세 브로슈어 요청
   - 포함 요청 항목: 기술 사양, 배터리 성능, 디자인 정보
5. 미팅 일정:
   - 날짜 및 시간: 1월 15일(화요일) 오전 10시
   - 장소: 귀사(티디인터내셔널) 사무실
6. 목적:
   - 유통 전략 및 마케팅 계획 구체화
   - 협력 가능성 논의


In [32]:
# Field의 description을 LLM이 읽어서 이해한다
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='메일 보낸 사람의 메일 주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')
    
parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())  # LLM에게 주는 지침

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일 보낸 사람의 메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [31]:
prompt = PromptTemplate.from_template("""
너는 요약 전문 어시스턴트야. 아래 질문과 형식에 맞게 답변을 한국어로 만들어줘.

---

질문: {question}

이메일 내용: {email_conversation}

형식: {format}

""")

# 일부 고정 변수
prompt = prompt.partial(format=parser.get_format_instructions())

In [35]:
chain = prompt | llm | parser
res = chain.invoke(
    {
        'question': '이메일 내용 중에 중요한 내용을 추출해 줘',
        'email_conversation' : email_conversation
    }
    )

In [ ]:
from pprint import pprint
pprint(res.model_dump())
# 앞으로 어떤 메일을 넣어도 지금처럼 형식을 맞춰 줄거라는 것

{'date': '2024년 1월 15일 오전 10시',
 'email': 'chulsoo.kim@bikecorporation.me',
 'person': '김철수 상무',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "바이크코퍼레이션 김철수 상무는 귀사의 'ZENESIS' 자전거에 대해 상세 브로슈어 요청과 함께, 기술 사양, 배터리 "
            '성능, 디자인 정보 확보를 희망하며, 다음 주 1월 15일 화요일 오전 10시에 사무실에서 협력 논의를 위한 미팅을 '
            '제안함.'}
